In [1]:
import open3d as o3d 
import numpy as np
from pathlib import Path
import scipy
import scipy.spatial as spt
import pcl
import math
from tqdm import tqdm
import itertools
import threading
import concurrent.futures
import functools

In [2]:
# get data base path to collect the point clouds
BASE = Path().resolve().parents[1]
DATA_PATH = BASE / 'data'
print(DATA_PATH)
SHOW_PC = True
CHUNKS = 5000
THRESHOLD = 0.5  # 5 cm
COLORS = {'green': [0, 255, 0], 'red': [255, 0, 0], 'blue': [0, 0, 255],
          'black': [0, 0, 0], 'yellow': [255, 255, 0]}

/home/dayoff/codes/point_cloud_lib/data


In [3]:
def calc_eucl_dist(p1, p2):
    return math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2 + (p2[2] - p1[2])**2)

In [4]:
def check_threshold(p1, p2):
    if ((p1[0] + THRESHOLD > p2[0]) and (p2[0] > p1[0] - THRESHOLD)) and \
        ((p1[1] + THRESHOLD > p2[1]) and (p2[1] > p1[1] - THRESHOLD)) and \
        ((p1[2] + THRESHOLD > p2[2]) and (p2[2] > p1[2] - THRESHOLD)):
        return calc_eucl_dist(p1, p2) <= THRESHOLD

In [5]:
def load_point_cloud(idx, color='blue'):
    # get first point cloud, assign it with a blue color
    filename = f'{idx}'.zfill(10) + '.ply'
    pc_path = DATA_PATH / 'kitti' / filename
    print('Collecting pc: ', pc_path)
    pcd = o3d.io.read_point_cloud(str(pc_path), format='ply')
    pcd_size = len(np.asarray(pcd.points))

    # define point cloud color
    np_colors = np.array([COLORS[color] for _ in range(pcd_size)])
    pcd.colors = o3d.utility.Vector3dVector(np_colors)

    pc_tot_size = len(point_clouds)
    # show point cloud info
    print(f'PCD {pc_tot_size}:\n', pcd, f'\nPCD {pc_tot_size} size: ', pcd_size,
          f'\n\nPCD {pc_tot_size} colors: \n', np.asarray(pcd.colors),
          f'\n\nPCD {pc_tot_size} points:\n', np.asarray(pcd.points))
    return pcd

In [6]:
def show_pc(pc_list):
    if SHOW_PC:
        new_pc_list = []
        pc_1 = pc_list[0]
        # define first point cloud color
        pc_1_size = len(np.asarray(pc_1.points))
        np_colors = np.array([COLORS['red'] for _ in range(pc_1_size)])
        pc_1.colors = o3d.utility.Vector3dVector(np_colors)
        new_pc_list.append(pc_1)

        if len(pc_list) == 2 or len(pc_list) == 3 or len(pc_list) == 4:
            pc_2 = pc_list[1]

            # offset the second point cloud
            offset_pc = o3d.geometry.PointCloud(pc_list[1])
            offset_pc.points = o3d.utility.Vector3dVector(
                np.asarray(pc_2.points) + [0.0, 0.0, -5.0])
            offset_pc_size = len(np.asarray(offset_pc.points))

            # define second point cloud color
            np_colors = np.array([COLORS['green'] for _ in range(offset_pc_size)])
            offset_pc.colors = o3d.utility.Vector3dVector(np_colors)
            
            new_pc_list.append(offset_pc)
        
        if len(pc_list) == 3 or len(pc_list) == 4:
            pc_3 = pc_list[2]

            # offset the third point cloud
            offset_pc = o3d.geometry.PointCloud(pc_3)
            offset_pc.points = o3d.utility.Vector3dVector(
                np.asarray(pc_3.points) + [0.0, 0.0, -10.0])
            offset_pc_size = len(np.asarray(offset_pc.points))

            # define the point cloud color
            np_colors = np.array([COLORS['blue'] for _ in range(offset_pc_size)])
            offset_pc.colors = o3d.utility.Vector3dVector(np_colors)

            new_pc_list.append(offset_pc)
                
        if len(pc_list) == 4:
            pc_4 = pc_list[3]

            # offset the third point cloud
            offset_pc = o3d.geometry.PointCloud(pc_4)
            offset_pc.points = o3d.utility.Vector3dVector(
                np.asarray(pc_4.points) + [0.0, 0.0, -15.0])
            offset_pc_size = len(np.asarray(offset_pc.points))

            # define the point cloud color
            np_colors = np.array([COLORS['black'] for _ in range(offset_pc_size)])
            offset_pc.colors = o3d.utility.Vector3dVector(np_colors)

            new_pc_list.append(offset_pc)

        o3d.visualization.draw_geometries(new_pc_list)

In [7]:
def sort_pc(pc):
    pc_np = np.asarray(pc.points)
    idx_sorted_pc = np.lexsort((pc_np[:,1], pc_np[:,0]))
    new_pc = o3d.geometry.PointCloud()
    new_pc.points = o3d.utility.Vector3dVector(pc_np[idx_sorted_pc])
    return new_pc

In [8]:
def get_min_max(pc):
    pc_np = np.asarray(pc.points)
    p_max, p_min = np.amax(pc_np, axis=0), np.amin(pc_np, axis=0)
    return p_max, p_min

In [9]:
def split_map(pc):
    pc_np = np.asarray(pc.points)
    new_pc_np = pc_np[(pc_np[:,0] >= -1.5) & (pc_np[:,0] <= 1.5)]
    new_pc = o3d.geometry.PointCloud()
    new_pc.points = o3d.utility.Vector3dVector(new_pc_np)
    print(f'Reduzed {len(pc_np)} point to {len(new_pc_np)} points.')
    return new_pc

In [10]:
def get_pc_from_pts_set(pc_set, pc):
    pts_list = list(pc_set)
    pc_np = np.asarray(pc.points) 
    pc_np_from_pts = pc_np[pts_list]

    new_pc = o3d.geometry.PointCloud()
    new_pc.points = o3d.utility.Vector3dVector(pc_np_from_pts)
    return new_pc

In [11]:
def get_diff_idx_pc(pc_set, pc):
    pts_list, diff_pts = list(pc_set), []
    pc_np = np.asarray(pc.points) 
    for idx, p in enumerate(pc_np):
        if idx not in pts_list:
            diff_pts.append(idx)
    pc_np_from_pts = pc_np[diff_pts]

    new_pc = o3d.geometry.PointCloud()
    new_pc.points = o3d.utility.Vector3dVector(pc_np_from_pts)
    return new_pc

In [12]:
def break_chunks(pc):
    pc_np = np.asarray(pc.points) 
    print(f'Received point cloud of size {len(pc_np)}')
    pc_np_size = len(pc_np)
    chunks, init, last = [], 0, CHUNKS
    while pc_np_size >= CHUNKS:
        pc_np_size -= CHUNKS
        chunk = pc_np[init:last]
        chunk_pc = o3d.geometry.PointCloud()
        chunk_pc.points = o3d.utility.Vector3dVector(chunk)
        chunks.append(chunk_pc)
        init = last
        last += CHUNKS
        print(f'Colleting chunk of {len(chunk)}')
    
    if pc_np_size > 0:
        chunk = pc_np[len(pc_np)-pc_np_size:]
        chunk_pc = o3d.geometry.PointCloud()
        chunk_pc.points = o3d.utility.Vector3dVector(chunk)
        chunks.append(chunk_pc)
        print(f'Colleting last chunk of {len(chunk)}')

    return chunks

In [13]:
point_clouds = []

In [14]:
pc = load_point_cloud(0, 'blue')
sorted_pc = sort_pc(pc)
# partial_pc = split_map(sorted_pc)
point_clouds.append(sorted_pc)
# show_pc([pc, partial_pc])

PCD 0:
 geometry::PointCloud with 120574 points. 
PCD 0 size:  120574 

PCD 0 colors: 
 [[  0.   0. 255.]
 [  0.   0. 255.]
 [  0.   0. 255.]
 ...
 [  0.   0. 255.]
 [  0.   0. 255.]
 [  0.   0. 255.]] 

PCD 0 points:
 [[74.536  9.937  2.752]
 [74.558 10.178  2.754]
 [74.569 10.419  2.755]
 ...
 [ 3.705 -1.394 -1.73 ]
 [ 3.611 -1.345 -1.681]
 [ 3.73  -1.377 -1.738]]


In [15]:
pc = load_point_cloud(10, 'red')
sorted_pc = sort_pc(pc)
# partial_pc = split_map(sorted_pc)
point_clouds.append(sorted_pc)
# show_pc([pc, partial_pc])

PCD 1:
 geometry::PointCloud with 122227 points. 
PCD 1 size:  122227 

PCD 1 colors: 
 [[255.   0.   0.]
 [255.   0.   0.]
 [255.   0.   0.]
 ...
 [255.   0.   0.]
 [255.   0.   0.]
 [255.   0.   0.]] 

PCD 1 points:
 [[78.601 10.984  2.893]
 [67.496  9.644  2.515]
 [67.487  9.859  2.515]
 ...
 [ 3.779 -1.43  -1.768]
 [ 3.809 -1.427 -1.78 ]
 [ 3.786 -1.405 -1.767]]


In [16]:
def kdtrees(pc_1, pc_2):
    pc_1_np, pc_2_np = np.asarray(pc_1.points), np.asarray(pc_2.points) 
    print('building first kdtree')
    pc1_kdtree = spt.cKDTree(pc_1_np)
    print('building second kdtree')
    pc2_kdtree = spt.cKDTree(pc_2_np)
    print('finished both kdtrees')
    return pc1_kdtree, pc2_kdtree

In [17]:
def kdtree_radius_search(pc_1, pc_2):
    pc_1_np, pc_2_np = np.asarray(pc_1.points, dtype=np.float32), np.asarray(pc_2.points, dtype=np.float32)
    cloud_1, cloud_2 = pcl.PointCloud(), pcl.PointCloud()
    cloud_1.from_array(pc_1_np)
    cloud_2.from_array(pc_2_np)
    kdtree = cloud_2.make_kdtree_flann()
    print(kdtree)
    indices, sqr_distances = kdtree.nearest_k_search_for_cloud(cloud_1, 1)
    print(indices, sqr_distances)
    return np.squeeze(sqr_distances), np.squeeze(indices)

In [18]:
idx, dist = kdtree_radius_search(point_clouds[0], point_clouds[1])

In [1]:
idx, dist

NameError: name 'idx' is not defined

In [2]:
idx, dist

NameError: name 'idx' is not defined

In [ ]:
pc1_kdtree, pc2_kdtree = kdtrees(point_clouds[0], point_clouds[1])

In [1]:
idx, dist

NameError: name 'idx' is not defined

In [33]:
dist_mat = pc2_kdtree.sparse_distance_matrix(pc1_kdtree, THRESHOLD, p=2.0, output_type='ndarray')

In [34]:
upper_dist_mat = dist_mat[dist_mat['i'] < dist_mat['j']]

In [35]:
upper_dist_mat

array([(  360,   397, 0.49465341), (  364,   397, 0.49785339),
       (  392,   407, 0.47970512), ..., (92039, 92104, 0.4919563 ),
       (92303, 92381, 0.47307293), (91725, 91967, 0.49037333)],
      dtype={'names':['i','j','v'], 'formats':['<i8','<i8','<f8'], 'offsets':[0,8,16], 'itemsize':24, 'aligned':True})

In [37]:
new_sparse_mat = scipy.sparse.coo_matrix((upper_dist_mat['v'], (upper_dist_mat['i'], upper_dist_mat['j'])), (125000, 125000))

In [38]:
new_sparse_mat

<125000x125000 sparse matrix of type '<class 'numpy.float64'>'
	with 2964531 stored elements in COOrdinate format>

In [40]:
cluster_pc = get_pc_from_pts_set(new_sparse_mat.toarray(), point_clouds[1])
off_cluster_pc = get_diff_idx_pc(new_sparse_mat.toarray(), point_clouds[1])

MemoryError: Unable to allocate 116. GiB for an array with shape (125000, 125000) and data type float64

In [32]:
cluster_pts = scipy_eucl_dist(point_clouds[0], point_clouds[1], True)

MemoryError: Unable to allocate 110. GiB for an array with shape (122227, 120574) and data type float64

In [21]:
cluster_pc = get_pc_from_pts_set(cluster_pts, point_clouds[1])
off_cluster_pc = get_diff_idx_pc(cluster_pts, point_clouds[1])

In [22]:
show_pc([point_clouds[0], point_clouds[1], cluster_pc, off_cluster_pc])